In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

In [ ]:
student_exp_name='lisa_fd7'
teacher_exp_name='0.001_lisa_offlineteacher_v1'
teacher_config='lstm_drop31_v2'
task_name = 'word_sv_agreement_lm'
student_model='lm_gpt2'
teacher_model='lm_lstm_shared_emb'
student_config='very_big_gpt_v10'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd10'
teacher_exp_name='0.001_lisa_offlineteacher_v1'
teacher_config='lstm_drop31_v2'
task_name = 'word_sv_agreement_lm'
student_model='lm_lstm_shared_emb'
teacher_model='lm_lstm_shared_emb'
student_config='lstm_drop31_v2'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd8'
teacher_exp_name='0.0001_lisa_offlineteacher_v1'
teacher_config='very_big_gpt_v10'
task_name = 'word_sv_agreement_lm'
student_model='lm_gpt2'
teacher_model='lm_gpt2'
student_config='very_big_gpt_v10'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
student_exp_name='lisa_fd9'
teacher_exp_name='0.0001_lisa_offlineteacher_v1'
teacher_config='very_big_gpt_v10'
task_name = 'word_sv_agreement_lm'
student_model='lm_lstm_shared_emb'
teacher_model='lm_gpt2'
student_config= 'lstm_drop31_v2'
distill_config='pure_distill_4'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [ ]:
task = TASKS[task_name](get_task_params(), data_dir='../data')

In [ ]:
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)
std_hparams=get_model_params(task, student_model, student_config)
std_hparams.output_attentions = True
std_hparams.output_embeddings = True
student_model = MODELS[student_model](
std_hparams, cl_token=cl_token)

In [ ]:
student_ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        teacher_config,
                                        teacher_exp_name,
                                       "student",student_model.model_name,
                                        str(student_config),
                                        student_exp_name]))
print("student_checkpoint:", student_ckpt_dir)

student_ckpt = tf.train.Checkpoint(net=student_model)
student_manager = tf.train.CheckpointManager(student_ckpt, student_ckpt_dir, max_to_keep=None)

student_ckpt.restore(student_manager.latest_checkpoint)
if student_manager.latest_checkpoint:
  print("Restored student from {}".format(student_manager.latest_checkpoint))

student_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

In [ ]:
teacher_ckpt_dir = os.path.join(chkpt_dir, task.name,
                                  '_'.join([teacher_model.model_name, teacher_config,teacher_exp_name]))

teacher_ckpt = tf.train.Checkpoint(net=teacher_model)
teacher_manager = tf.train.CheckpointManager(teacher_ckpt, teacher_ckpt_dir, max_to_keep=None)

teacher_ckpt.restore(teacher_manager.latest_checkpoint)
if teacher_manager.latest_checkpoint:
  print("Restored student from {}".format(teacher_manager.latest_checkpoint))

teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

In [ ]:
train = teacher_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = teacher_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = teacher_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [ ]:
train = student_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = student_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = student_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

In [ ]:
model_name='lm_gpt2'
model_config='big_gpt_v5'
learning_rate=0.0001
exp_name='lisa_offlineteacher_v3'
task_name = 'word_sv_agreement_lm'
chkpt_dir = '../tf_ckpts'

task = TASKS[task_name](get_task_params(), data_dir='../data')

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True

model = MODELS[model_name](hparams=hparams, cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir,task.name,
                        model.model_name+"_"+str(model_config)+"_"+str(learning_rate)+"_"+exp_name)

ckpt = tf.train.Checkpoint(net=model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))
else:
 print("Initialized from scratch")
 print(ckpt_dir)

model.compile(loss=task.get_loss_fn(), metrics=task.metrics())



train = model.evaluate(task.train_dataset, steps=task.n_train_batches)
print("train:", train)

valid = model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
print("valid:", valid)

test = model.evaluate(task.test_dataset, steps=task.n_test_batches)
print("test:", test)

Vocab len:  10034
model config: big_gpt_v5
{'embedding_dim': 256, 'resid_pdrop': 0.2, 'embd_pdrop': 0.2, 'attn_pdrop': 0.3, 'initializer_range': 0.05}
Restored student from ../tf_ckpts/word_sv_agreement_lm/lm_gpt2_h-256_d-6_rdrop-0.2_adrop-0.3_indrop-0.2_big_gpt_v5_0.0001_lisa_offlineteacher_v3/ckpt-60


2464/2464 [==============================] - 168s 68ms/step - loss: 3.7552 - batch_masked_sequence_loss: 3.5545 - masked_batch_perplexity: 35.1102 - masked_perplexity: 41.8895 - accuracy: 0.0784 - accuracy_top2: 0.1013 - accuracy_top5: 0.1301
train: [3.7552120925544146, 3.5545235, 35.110172, 41.88948, 0.07842734, 0.10127362, 0.13008937]
246/246 [==============================] - 17s 69ms/step - loss: 4.2163 - batch_masked_sequence_loss: 4.0156 - masked_batch_perplexity: 55.6864 - masked_perplexity: 72.1779 - accuracy: 0.0730 - accuracy_top2: 0.0946 - accuracy_top5: 0.1218
valid: [4.216335023321757, 4.0156455, 55.686382, 72.17794, 0.07301878, 0.09457148, 0.12181229]
 8480/21933 [==========>...................] - ETA: 15:31 - loss: 4.2137 - batch_masked_sequence_loss: 4.0130 - masked_batch_perplexity: 55.5691 - masked_perplexity: 72.1284 - accuracy: 0.0725 - accuracy_top2: 0.0939 - accuracy_top5: 0.1208